# Part 1.2 - Extract Predictions for RNN 
In this notebook, we will load a pre-trained RNN model and run both our train and test samples through it, extracting the bottleneck features (the outputs of the layer just before the fully-connected / classification layer(s)). 

These features will be used to represent the embedded timeseries features for each sample. They will be concatenated with other features to train our final XGBoost classification model. 

In [1]:
import warnings
warnings.filterwarnings("ignore")
import math
import pandas as pd
import numpy as np
import time
import tensorflow as tf
from rnn import PlasticcRNN
import matplotlib.pyplot as plt
%matplotlib inline

print(tf.__version__)

1.13.1


### Load Train & Test Data

In [2]:
train = pd.read_pickle('train_rnn.pkl')
test = pd.read_pickle('test_rnn.pkl')

### Load pre-trained RNN model

Set relevant parameters and load the model. You can find the code for this model in `rnn.py` if you'd like to look further into the RNN implementation.

In [3]:
model = PlasticcRNN('weight/rnn.npy')

Call `predict_bottleneck` to feed each training example through the pre-trained RNN model and extract the outputs from the layer just before the final classification layer .

In [4]:
train_bn = model.predict_bottleneck(train)

Instructions for updating:
Colocations handled automatically by placer.

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This class is equivalent as tf.keras.layers.GRUCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
restore RNN/rnn3/bidirectional_rnn/fw/gru_cell/gates/kernel:0
restore RNN/rnn3/bidirectional_rnn/fw/gru_cell/gates/bias:0
restore RNN/rnn3/bidirectional_rnn/fw/gru_cell/candidate/kernel:0
restore RNN/rnn3/bidirectional_rnn/fw/gru_cell/candidate/bias:0
restore RNN/rnn3/bidirectional_rnn/bw/gru_cell/gates/kernel:0
restore RNN/rnn3/bidirectional

  0%|          | 0/3 [00:00<?, ?it/s]

restore RNN/rnn4/bidirectional_rnn/bw/gru_cell/gates/kernel:0
restore RNN/rnn4/bidirectional_rnn/bw/gru_cell/gates/bias:0
restore RNN/rnn4/bidirectional_rnn/bw/gru_cell/candidate/kernel:0
restore RNN/rnn4/bidirectional_rnn/bw/gru_cell/candidate/bias:0


4it [00:03,  1.00it/s]                       


Call `predict_bottleneck` to do the same with the testing data. This can take a little time, so it might be worthwhile to move onto the next nextbook and return to this once it's complete. 

In [5]:
test_bn = model.predict_bottleneck(test)

restore RNN/rnn3/bidirectional_rnn/fw/gru_cell/gates/kernel:0
restore RNN/rnn3/bidirectional_rnn/fw/gru_cell/gates/bias:0
restore RNN/rnn3/bidirectional_rnn/fw/gru_cell/candidate/kernel:0
restore RNN/rnn3/bidirectional_rnn/fw/gru_cell/candidate/bias:0
restore RNN/rnn3/bidirectional_rnn/bw/gru_cell/gates/kernel:0
restore RNN/rnn3/bidirectional_rnn/bw/gru_cell/gates/bias:0
restore RNN/rnn3/bidirectional_rnn/bw/gru_cell/candidate/kernel:0
restore RNN/rnn3/bidirectional_rnn/bw/gru_cell/candidate/bias:0
restore RNN/rnn5/bidirectional_rnn/fw/output_projection_wrapper/gru_cell/gates/kernel:0
restore RNN/rnn5/bidirectional_rnn/fw/output_projection_wrapper/gru_cell/gates/bias:0
restore RNN/rnn5/bidirectional_rnn/fw/output_projection_wrapper/gru_cell/candidate/kernel:0
restore RNN/rnn5/bidirectional_rnn/fw/output_projection_wrapper/gru_cell/candidate/bias:0
restore RNN/rnn5/bidirectional_rnn/fw/output_projection_wrapper/kernel:0
restore RNN/rnn5/bidirectional_rnn/fw/output_projection_wrapper/bia

722it [07:46,  2.11it/s]                         


Let's verify that we have embedded each of our timeseries into 16-dimensional space. 

In [6]:
print(train_bn.shape)
print(test_bn.shape)

(7848, 16)
(1477022, 16)


### Convert Bottleneck Features to DataFrames

In [7]:
train_bn = pd.DataFrame(train_bn,columns=['bottleneck%d'%i for i in range(train_bn.shape[1])])
train_bn['object_id'] = train.object_id.unique()

In [8]:
train_bn.head()

,bottleneck0,bottleneck1,bottleneck2,bottleneck3,bottleneck4,bottleneck5,bottleneck6,bottleneck7,bottleneck8,bottleneck9,bottleneck10,bottleneck11,bottleneck12,bottleneck13,bottleneck14,bottleneck15,object_id
0,39.626457,0.497825,9.503530,0.612190,0.370296,36.647839,0.001450,0.050136,0.076503,7.458235,0.147237,0.000974,0.472516,56.940910,0.144692,20.955284,615
1,2.315192,6.362340,8.646493,0.750931,2.506892,8.227393,7.570269,9.884037,3.889832,1.148435,2.220537,0.156296,5.254804,1.756874,3.062057,19.185877,713
2,2.349471,29.854164,0.534167,1.965333,7.866547,0.414091,21.975964,3.617615,3.761395,2.100560,0.536902,11.514246,3.766770,0.182644,9.025029,3.807947,730
3,14.263150,32.955151,3.364045,11.658510,2.128215,0.905252,25.994488,2.800233,9.684278,8.299098,0.385034,8.098384,4.981500,0.281322,15.864852,2.367192,745
4,9.047191,27.102867,8.031672,6.901597,2.862491,1.167179,23.666925,1.425559,7.036583,7.211462,0.203276,9.391024,2.969179,0.419447,8.057733,1.478815,1124


In [9]:
test_bn = pd.DataFrame(test_bn,columns=['bottleneck%d'%i for i in range(test_bn.shape[1])])
test_bn['object_id'] = test.object_id.unique()

In [10]:
test_bn.head()

,bottleneck0,bottleneck1,bottleneck2,bottleneck3,bottleneck4,bottleneck5,bottleneck6,bottleneck7,bottleneck8,bottleneck9,bottleneck10,bottleneck11,bottleneck12,bottleneck13,bottleneck14,bottleneck15,object_id
0,0.951065,0.251945,3.318633,0.000736,2.441339,2.927944,1.147542,4.429283,3.313076,0.003929,1.196651,0.306426,1.503734,0.039145,0.131615,0.641174,49433749
1,11.885058,2.603644,3.270815,1.826216,1.102672,1.243358,3.490134,7.530881,10.645997,0.500468,6.901350,4.232432,2.833216,0.686619,8.815798,5.550112,49433769
2,0.660288,8.012924,0.600806,2.113838,3.681345,0.808573,6.829436,2.589654,5.121986,1.699079,4.928893,2.138904,1.987519,0.085826,4.800367,4.929393,49433826
3,7.590333,5.189585,0.883393,0.451710,1.230645,6.413009,7.873522,6.185088,3.346524,1.267524,5.761439,7.788362,5.899473,0.721337,10.079144,8.565522,49433842
4,2.929134,8.943206,1.777365,0.801600,0.625757,1.670301,2.025340,1.249137,11.079231,3.141696,1.676079,8.165009,3.502824,4.667575,10.525523,4.981074,49433919


### Store Features to Disk

In [11]:
train_bn.to_pickle('train_bn.pkl')
test_bn.to_pickle('test_bn.pkl')